In [ ]:
#
#    Notebook de cours AMS-X02 - Cours 2 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Exemples de systèmes dynamiques
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

# Exemples de systèmes dynamiques

## Modèle de Brusselator

La dynamique de la réaction oscillante découverte par Belousov et Zhabotinsky, peut être modélisée par le modèle dit de Brusselator en fonction de deux paramètres $a$ et $b$ :

$$
\left\{\begin{aligned}
{\mathrm d_t} y_1 & = a - (b+1) y_1 + y_1^2y_2\\
{\mathrm d_t} y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
class brusselator_model:

    def __init__(self, a, b) :
        self.a = a
        self.b = b
    
    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b 
        y1_dot = a - (b+1)*y1 + y1*y1*y2 
        y2_dot = b*y1 - y1*y1*y2  
        return np.array([y1_dot, y2_dot])

### Solution numérique

In [ ]:
tini = 0. 
tend = 40.
yini = (1.5, 3.)

bm = brusselator_model(a=0.7, b=3)
fcn = bm.fcn  
sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-10, atol=1.e-10)

fig = make_subplots(rows=2, cols=1, subplot_titles=("Espace des phases", "Solutions"), vertical_spacing=0.15)
fig.add_trace(go.Scatter(x=sol_exa.y[0], y=sol_exa.y[1], mode="lines", showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[0], mode="lines", name="y1"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[1], mode="lines", name="y2"), row=2, col=1)

steps = []
for i, a_i in enumerate(np.arange(0.7, 1.71, 0.1)):
    bm = brusselator_model(a=a_i, b=3)
    fcn = bm.fcn  
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-10, atol=1.e-10)
    step = dict(method="update", label = f"{a_i:.2f}", args=[{"x":[sol_exa.y[0], sol_exa.t, sol_exa.t],
                                                              "y":[sol_exa.y[1], sol_exa.y[0], sol_exa.y[1]]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'a = '}, steps=steps)]

fig.update_xaxes(title_text="u1", range=[0, 6], row=1)
fig.update_yaxes(title_text="u2", row=1)
fig.update_xaxes(title_text="t", row=2)
fig.update_layout(sliders=sliders, height=800, legend=dict(x=0.01, y=0.37, bgcolor='rgba(0,0,0,0.1)'))
fig['layout']['sliders'][0]['pad']=dict(t= 50)
fig.show()

## Système de Lorenz

Le système de Lorenz est un système de trois équations différentielles ordinaires : 

$$
\left\{\begin{aligned}
{\mathrm d_t} x & = \sigma \, (y - x)\\
{\mathrm d_t} y & = \rho x - y - xz\\ 
{\mathrm d_t} z & = xy - \beta z
\end{aligned}\right.
$$

In [ ]:
class lorenz_model():

    def __init__(self, sigma, rho, beta):
        self.sigma = sigma
        self.rho = rho
        self.beta = beta

    def fcn(self, t, xyz):
        x, y, z = xyz
        sigma = self.sigma
        rho = self.rho
        beta = self.beta
        x_dot = sigma*(y-x)
        y_dot = rho*x - y - x*z
        z_dot = x*y - beta*z
        return (x_dot, y_dot, z_dot)

### Solution numérique

In [ ]:
lm = lorenz_model(sigma=10, rho=28, beta=8/3)
fcn = lm.fcn  
    
tini = 0. 
tend = 100.
    
sol_ini = (-10, -7, 35)        
sol = solve_ivp(fcn, (tini, tend), sol_ini, method="RK45", rtol=1.e-8, atol=1.e-10)
x = sol.y[0]; y = sol.y[1]; z = sol.y[2]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode="lines"))
fig.update_layout(scene_camera=dict(eye=dict(x=-1.3, y=1.3, z=1.3)), title="Espace des phases")
fig.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=x, mode="lines", name="evolution de la variable x"))
fig_sol.update_layout(title="Evolution de la variable x")
fig_sol.show()

### Perturbations des conditions initiales

In [ ]:
sol_ini_pert = (-10.000000001, -7, 35)

sol_pert = solve_ivp(fcn, (tini, tend), sol_ini_pert, method="RK45", rtol=1.e-8, atol=1.e-10)
x_pert = sol_pert.y[0]; y_pert = sol_pert.y[1]; z_pert = sol_pert.y[2]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode="lines", name="sol."))
fig.add_trace(go.Scatter3d(x=x_pert, y=y_pert, z=z_pert, mode="lines", name = "sol. perturbée"))
fig.update_layout(scene_camera=dict(eye=dict(x=-1.3, y=1.3, z=1.3)), title="Espace des phases")
fig.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=x, mode="lines", name="sol."))
fig_sol.add_trace(go.Scatter(x=sol_pert.t, y=x_pert, mode="lines", name="sol. perturbée"))
fig_sol.update_layout(title="Evolution de la variable x")
fig_sol.show()

## Mécanique céleste : problème à trois corps

On considère le système composé de trois corps : le Soleil, Jupiter et Saturne, les trois astres les plus massifs du système solaire. On fait l’hypothèse que le système est isolé et que les seules forces qui s’exercent sont les forces de gravitation entre les trois corps. 

Les masses respectives des trois corps soit noteés $m_i$, $i\in{0,1,2}$ et $G$ désigne la constante universelle de gravitation. On note $p_i(t)\in R^3$, $i\in{0,1,2}$ les quantités de mouvement de chaque astre. 

Le second principe de Newton pour chaque astre s'écrit :

$$
{\mathrm d}_t p_0 = \vec F_{Sa\rightarrow S} + \vec F_{J\rightarrow S},\quad
{\mathrm d}_t p_1 = \vec F_{S\rightarrow J} + \vec F_{Sa\rightarrow J} ,\quad
{\mathrm d}_t p_2 = \vec F_{S\rightarrow Sa} + \vec F_{J\rightarrow Sa},
$$

La force gravitationnelle $\vec F_{S\rightarrow P}$ exercée par un corps $S$ vers un corps $P$ est donné par la formule suivante :

$$
\vec F_{S\rightarrow P} = - \vec F_{P\rightarrow S} = - \frac{G\,m_S\,m_P}{d^2} \vec{u}, 
$$

où $d$ est la distance entre $S$ à $P$ et $\vec{u}$ un vecteur de longueur 1 dirigé de S vers P. 

In [ ]:
class three_body_model:

    def __init__(self):
        self.m0 = 1.00000597682
        self.m1 = 9.54786104043e-4
        self.m2 = 2.85583733151e-4
        self.G = 2.95912208286e-4

    def fcn(self, t, y):
        q = y[0:9]
        p = y[9:18]
        
        q_dot = np.zeros(9)
        p_dot = np.zeros(9)
        
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G  = self.G
        
        q_dot[0:3] = p[0:3]/m0
        q_dot[3:6] = p[3:6]/m1
        q_dot[6:9] = p[6:9]/m2
        
        p_dot[0:3] = -( G*m0*m1*((q[0:3]-q[3:6])/np.power(np.linalg.norm(q[0:3]-q[3:6]),3)) 
                       +G*m0*m2*((q[0:3]-q[6:9])/np.power(np.linalg.norm(q[0:3]-q[6:9]),3)))
        p_dot[3:6] = -( G*m1*m0*((q[3:6]-q[0:3])/np.power(np.linalg.norm(q[3:6]-q[0:3]),3))
                       +G*m1*m2*((q[3:6]-q[6:9])/np.power(np.linalg.norm(q[3:6]-q[6:9]),3)))
        p_dot[6:9] = -( G*m2*m0*((q[6:9]-q[0:3])/np.power(np.linalg.norm(q[6:9]-q[0:3]),3)) 
                       +G*m2*m1*((q[6:9]-q[3:6])/np.power(np.linalg.norm(q[6:9]-q[3:6]),3)))
        
        return np.concatenate((q_dot, p_dot))
    
    def hamiltonian(self, y):
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        nt = y.shape[1]
        neq = y.shape[0]
        q = y[0:neq//2]
        p = y[neq//2:neq]
        ham = np.zeros(nt)

        for i in range(nt):
            ham[i] = ( 0.5*( (1/m0)*np.dot(p[0:3,i],p[0:3,i]) 
                            +(1/m1)*np.dot(p[3:6,i],p[3:6,i]) 
                            +(1/m2)*np.dot(p[6:9,i],p[6:9,i]) ) 
                      - G *( (m0*m1)/np.linalg.norm(q[0:3,i]-q[3:6,i]) 
                            +(m0*m2)/np.linalg.norm(q[0:3,i]-q[6:9,i])
                            +(m1*m2)/np.linalg.norm(q[3:6,i]-q[6:9,i]) ) )

        return ham

In [ ]:
# initialization
m1 = 9.54786104043e-4
m2 = 2.85583733151e-4

qini = np.zeros(9)
qini[0] =  0.;        qini[1] =  0.;        qini[2] =  0.
qini[3] = -3.5023653; qini[4] = -3.8169847; qini[5] = -1.5507963
qini[6] =  9.0755314; qini[7] = -3.0458353; qini[8] = -1.6483708

pini = np.zeros(9)
pini[0] =  0.;            pini[1] =  0.;            pini[2] =  0.
pini[3] =  0.00565429*m1; pini[4] = -0.00412490*m1; pini[5] = -0.00190589*m1
pini[6] =  0.00168318*m2; pini[7] =  0.00483525*m2; pini[8] =  0.00192462*m2

yini = np.concatenate((qini, pini))

tini = 0.
tend = 11000.
nt = 1501

tbm = three_body_model()
fcn = tbm.fcn

sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

fig = go.Figure()
color = fig.layout.template.layout.colorway
fig.add_trace(go.Scatter3d(x=[qini[0]], y=[qini[1]], z=[qini[2]], mode="markers", marker=dict(size=8, color=color[1]), 
                           name="soleil"))
fig.add_trace(go.Scatter3d(x=[qini[3]], y=[qini[4]], z=[qini[5]], mode="markers", 
                           marker=dict(size=6, color=color[0]), showlegend=False))
fig.add_trace(go.Scatter3d(x=sol.y[3]-sol.y[0], y=sol.y[4]-sol.y[0], z=sol.y[5]-sol.y[0], mode='lines', 
                           line_color=color[0], line_width=6, name='Jupiter'))
fig.add_trace(go.Scatter3d(x=[qini[6]], y=[qini[7]], z=[qini[8]], mode="markers", 
                           marker=dict(size=6, color=color[2]), showlegend=False))
fig.add_trace(go.Scatter3d(x=sol.y[6]-sol.y[0], y=sol.y[7]-sol.y[0], z=sol.y[8]-sol.y[0], mode='lines', 
                           line_color=color[2], line_width=6, name='Saturne'))
fig.show()

### Hamiltonien

In [ ]:
ham = tbm.hamiltonian(sol.y)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.t, y=ham))
fig.update_layout(title="Hamiltonian")
fig.update_yaxes(range=[-3.157e-8, -3.156e-8], exponentformat='e')
fig.show()

## L'orbite d'Arenstorf

Nous considérons un problème réduit à trois corps constitué du mouvement d'un satellite dans le cadre de l'attraction de la lune et de la terre. 
Pour les besoins de l'exercice, nous supposons que le système terre-lune est isolé et en rotation circulaire à vitesse constante dans un mouvement plan avec le centre de gravité de masse situé à l'origine, une solution classique d'une orbite périodique circulaire en mécanique céleste de deux corps isolés (appelé problème à deux corps classiquement).
Nous supposons aussi que la masse du satellite $\epsilon$ est suffisamment faible par rapport à la masse de la terre $1-\mu$ et la masse de la lune $\mu$ pour que nous puissions négliger son impact sur le système terre-lune.  Nous supposons également que le mouvement du satellite est régi par la loi de gravitation de Newton du fait de l'attraction des deux corps Terre et Lune.

Le mouvement du satellite dans le plan complexe satisfait l'équation : 

$$
\epsilon\, {\mathrm d}_t^2 Y = \frac{\epsilon(1-\mu)}{\vert A-Y \vert^2}\, \frac{A-Y}{\vert A-Y\vert}+ \frac{\epsilon\mu}{\vert B-Y\vert^2}\, \frac{B-Y}{\vert B-Y\vert},
$$

où le satellite est repéré par la coordonnée dans le plan complexe $Y(t)$ et $A$ et $B$ représentent la position de la terre et de la lune respectivement.

Pour éliminer le facteur $e^{it}$ dans $A=-\mu\,e^{it}$ et $A=(1-\mu)e^{it}$, on introduit la variable $y=e^{-it}\,Y = y_1+i\,y_2$. Dans ce nouveau référentiel tournant, la terre et la lune sont immobiles. Nous avons $Y=e^{it}y$ et  ${\mathrm d}_t^2 Y = -e^{it}y+2\,i\,e^{it}{\mathrm d}_t y+e^{it}{\mathrm d}_t^2 y$ et l'équation du mouvement s'écrit alors :

$$
{\mathrm d}_t^2 y + 2\,i\,{\mathrm d}_t y-y= (1-\mu)\,\frac{-\mu-y}{\vert  \mu+y\vert ^3}+ \mu\, \frac{1-\mu-y}{\vert  1-\mu-y \vert^3}.
$$

En introduisant les parties réelles et imaginaires de $y$ et en passant ensuite à un système d'équations différentielles de premier ordre, on obtient :

$$
\begin{array}{rcl} 
{\mathrm d}_t y_1 & = & y_3, \\ 
{\mathrm d}_t y_2 & = & y_4, \\ 
{\mathrm d}_t y_3 & = & y_1+2\,y_4- (1-\mu)(y_1+\mu)/r_1^3 - \mu (y_1-1+\mu)/r_2^3, \\
{\mathrm d}_t y_4 & = & y_2-2\,y_3- (1-\mu)y_2/r_1^3 - \mu y_2/r_2^3,
\end{array} 
$$

avec $r_1=((y_1+\mu)^2+y_2^2)^{1/2}$ and $((y_1-1+\mu)^2+y_2^2)^{1/2}$ et $\mu = 0.012277471$.

In [ ]:
class arenstorf_model:

    def __init__(self, mu):
        self.mu = mu

    def fcn(self, t, y) :
        y1,y2,y3,y4 = y
        mu = self.mu
        r1 = np.sqrt((y1+mu)*(y1+mu) + y2*y2)
        r2 = np.sqrt((y1-1+mu)*(y1-1+mu) + y2*y2)
        y1_dot = y3
        y2_dot = y4
        y3_dot = y1 + 2*y4 - (1-mu)*(y1+mu)/(r1*r1*r1) - mu*(y1 - 1 + mu)/(r2*r2*r2)
        y4_dot = y2 - 2*y3 - (1-mu)*y2/(r1*r1*r1) - mu*y2/(r2*r2*r2)
        return (y1_dot, y2_dot, y3_dot, y4_dot)

In [ ]:
# Initialization
yini = (-0.5655899165951338, 0.601315396569226, -0.45171183358756384, 0.23073427996775764)
#yini = (0.994, 0., 0., -2.00158510637908252240537862224)

tini = 0.
tend = 18.

mu = 0.012277471
    
am = arenstorf_model(mu)
fcn = am.fcn

tol = 1.e-12
sol = solve_ivp(fcn, (tini, tend), yini, rtol=1e-10, atol=1e-10)

fig_y1y2 = go.Figure()
fig_y1y2.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol.", showlegend=True))
fig_y1y2.add_trace(go.Scatter(x=[mu], y=[0.], mode="markers", marker=dict(size=10, color="black"), name="Terre"))
fig_y1y2.add_trace(go.Scatter(x=[1-mu], y=[0], mode="markers", marker=dict(size=6, color="brown"), name="Lune"))
fig_y1y2.update_layout(title="Orbite d'Arenstorf", xaxis_title="y1", yaxis_title="y2")
fig_y1y2.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name="y1"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name="y2"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[2], name="y3"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[3], name="y4"))
fig_sol.update_layout(title="Evolution des solutions", xaxis_title="t")
fig_sol.show()

## Dynamique des populations

### Modèled de Lotka-Voltera

A.J. Lotka publie en 1925 un livre intitulé `Elements of Physical Biology` , où il se propose de représenter les cinétiques de populations vivant en communauté par des systèmes d'équations différentielles.
Dans l'un des chapitres, il considère l'exemple d'une population d'animaux herbivores qui se nourrissent de plantes. Par analogie avec les équations utilisées pour la cinétique chimique, en représentant par $u_1(t)$ la masse totale des plantes et par $u_2(t)$ la masse totale des herbivores à l'instant $t$, Lotka propose le modèle suivant :

$$
\left\{\begin{aligned}
{\mathrm d}_t u_1 & = u_1 \, (1-u_2)\\
{\mathrm d}_t u_2 & = u_2 \, (-k+u_1).
\end{aligned}\right.
$$

In [ ]:
class lotka_model:

    def __init__(self, alpha):
        self.alpha = alpha

    def fcn(self, t, u):
        u1, u2 = u
        alpha = self.alpha
        u1_dot = u1 * (1 - u2)
        u2_dot = u2 * (-alpha + u1)
        return (u1_dot, u2_dot)

In [ ]:
alpha = np.arange(0.5,2.6,0.2)

lm = lotka_model(alpha[0])
fcn = lm.fcn

uini = (0.5, 0.5)
tini = 0
tend = 200

tol = 1.e-8

sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
u1 = sol.y[0]; u2 = sol.y[1]
inv = u1 + u2 - np.log(u2 * np.power(u1,alpha[0]))

fig = make_subplots(rows=2, cols=1, subplot_titles=("Espace des phases", "Solutions"), vertical_spacing=0.15)
fig.add_trace(go.Scatter(x=u1, y=u2, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u1, name="u1"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u2, name="u2"), row=2, col=1)

#create slider
steps = []
for alpha_i in alpha:
    lm = lotka_model(alpha_i)
    fcn = lm.fcn
    sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
    u1 = sol.y[0]; u2 = sol.y[1]
    inv = u1 + u2 - np.log(u2 * np.power(u1,alpha_i))
    step = dict(method="update", label = f"{alpha_i:.2f}", args=[{"x": [u1,sol.t,sol.t], "y": [u2, u1, u2]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha = '}, steps=steps)]

fig.update_xaxes(title_text="u1", range=[0, 8], row=1)
fig.update_yaxes(title_text="u2", range=[0, 5], row=1)
fig.update_xaxes(title_text="t", row=2)
fig.update_yaxes(title_text="t", range=[0, 8], row=2)
fig.update_layout(sliders=sliders, height=800, legend=dict(x=0.01, y=0.37, bgcolor='rgba(0,0,0,0.1)'))
fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()

### Rosenzweig-MacArthur model

Rapidement après l'introduction du modèle de Lotka-Volterra, diverses modifications ont été proposées pour le terme de prédation. En effet, le nombre de proies tuées par les prédateurs est dans ce modèle proportionnel au produit du nombre d'individus de chaque population, i.e. proportionnel à $u_1(t)u_2(t)$. Autrement dit, le nombre de proies tuées par prédateur croit proportionnellement au nombre de proies lui-même et il n'y a donc aucun effet de saturation ou de satiété. C'est qualitativement ce qu'on observe pour certaines populations de bactéries. Cependant, pour des organismes vivants plus complexes, comme des animaux, on s'attend à un comportement vraiment différent, où les prédateurs ne peuvent pas consommer plus qu'une quantité maximale de proies par unité de temps.

L'écologue américain Buzz Holling a proposé en 1959 trois grands types de modélisation du nombre de proies tuées par
prédateur : la première est celle du modèle de Lotka-Volterra (type I) et les deux autres introduisent un effet de saturation lorsque le nombre de proies dépasse un certain seuil (types II et III). Les types II et III diffèrent quand le nombre de proies est très petit et
permettent de distinguer les prédateurs *généralistes* des prédateurs *spécialistes*.

C'est en 1963 que les écologues américains Robert MacArthur et Michael L. Rosenzweig étudièrent le modèle proie-prédateur suivant :

$$
\left\{\begin{aligned}
{\mathrm d}_t u_1  & = u_1 \, \Big( 1-\frac{u_1}{\gamma} \Big) - \frac{u_1 u_2}{1+u_1}\\
{\mathrm d}_t u_2  & = \beta u_2 \Big( \frac{u_1}{1+u_1} - \alpha \Big).  
\end{aligned}\right.
$$

Le terme de prédation est de type II.

In [ ]:
class rosenzweig_model:

    def __init__(self, alpha, K):
        self.alpha = alpha
        self.K = K

    def fcn(self, t, u):
        u1, u2 = u
        alpha = self.alpha
        K = self.K
        u1_dot = u1 * (1 - (u1/K)) - ((u1*u2)/(1+u1))
        u2_dot = u2 * ((u1/(1+u1)) - alpha)
        return (u1_dot, u2_dot)

In [ ]:
alpha = np.arange(0.2,0.65,0.05)
K = 2
rm = rosenzweig_model(alpha[0], K)
fcn = rm.fcn

uini = (1., 1.)
tini = 0
tend = 200

tol = 1.e-8

sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
u1 = sol.y[0]; u2 = sol.y[1]

fig = make_subplots(rows=2, cols=1, subplot_titles=("Espace des phases", "Solutions"), vertical_spacing=0.15)
fig.add_trace(go.Scatter(x=u1, y=u2, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u1, name="u1", mode="lines"), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=u2, name="u2", mode="lines"), row=2, col=1)

#create slider
steps = []
for alpha_i in alpha:
    rm = rosenzweig_model(alpha_i, K)
    fcn = rm.fcn
    sol = solve_ivp(fcn, (tini, tend), uini, rtol=tol, atol=tol)
    u1 = sol.y[0]; u2 = sol.y[1]
    step = dict(method="update", label = f"{alpha_i:.2f}", args=[{"x": [u1,sol.t,sol.t], "y": [u2, u1, u2]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'alpha = '}, steps=steps)]

fig.update_xaxes(title_text="u1", range=[-0.1, 2.5], row=1)
fig.update_yaxes(title_text="u2", range=[-0.1, 2.5], row=1)
fig.update_xaxes(title_text="t", row=2)
fig.update_yaxes(range=[-0.1, 2.5], row=2)
fig.update_layout(sliders=sliders, height=800, legend=dict(x=0.88, y=0.37, bgcolor='rgba(0,0,0,0.1)'))
fig['layout']['sliders'][0]['pad']=dict(t=50)

fig.show()